# Set up

In [1]:
#import packages

import pandas as pd
from nilearn import surface
import numpy as np
import os
from scipy.stats import ttest_1samp
from nilearn import plotting
import matplotlib.pyplot as plt
from nilearn.datasets import fetch_surf_fsaverage
import nibabel as nib
import usefulFunctions as uf

In [4]:
# flag to save curv files
save_maps = False

#set paths
base_dir = f'{os.path.dirname(os.getcwd())}/'
c_map_dir = f'{base_dir}data/c_maps/average/allData/'
out_dir = f'{base_dir}analysis/group_maps/'
fig_dir = f'{base_dir}figures/'

#define sub list
subs = uf.subject_list(c_map_dir) # grab all sub IDs
#separate sub list into adults and children
id_cutoff = 1000 # split between child and adult ID #s
child_subs = [sub for sub in subs if int(sub.split('-')[1]) < id_cutoff]
adult_subs = [sub for sub in subs if int(sub.split('-')[1]) >= id_cutoff]
sub_groups = [child_subs,adult_subs]
#specify contrasts
contrasts = ['T-v-ALL','F-v-ALL','FF-v-ALL','O-v-ALL','L-v-ALL'] # list of contrasts
#specify group
group_names = ['children','adults']



# Calculate group average maps

In [5]:
# collect data
all_t_maps = []
all_p_maps = []

# loop through group and contrast to calculate second level model
for group_indx, group in enumerate(sub_groups):
    group_t_maps = []
    group_p_maps = []
    for contrast in contrasts:
        c_maps = []
        for sub in group:

            # load individual contrast maps
            c_map = surface.load_surf_data(f'{c_map_dir}{sub}/{sub}_{contrast}_LH.curv')
            c_maps.append(c_map)

        # run a 1 sample t test on all contrast maps
        t_map, p_map = ttest_1samp(np.array(c_maps), 0)
        group_t_maps.append(t_map)
        # group_p_maps.append(p_map)

        # save the resulting group average map
        if save_maps:
            nib.freesurfer.io.write_morph_data(f'{out_dir}{group_names[group_indx]}_{contrast}_t-map.curv', t_map)
            # nib.freesurfer.io.write_morph_data(f'{out_dir}{group_names[group_indx]}_{contrast}_p-map.curv', p_map)
    
    # store group maps for plotting
    all_t_maps.append(group_t_maps)
    # all_p_maps.append(group_p_maps)
        

# Plot group average maps

In [4]:
# define paths
map_dir = out_dir
surf = f'{base_dir}data/surface_meshes/fsaverage/lh.inflated'
ss_dir = f'{fig_dir}group_activations/'

# define variables
groups=['adults','children']
contrasts = ['T-v-ALL','F-v-ALL','FF-v-ALL','O-v-ALL','L-v-ALL']
viewsize = '600 665'
view = 'inferior'
vmax = 9
vmin = 3

In [5]:
# visualizations with freeview

# loop through groups and contrasts to grab freeview screenshots of group maps
for group in groups:
    for contrast in contrasts:
        cmd = f'\nfreeview -f {surf}:overlay={map_dir}{group}_{contrast}_t-map.curv:overlay_threshold={vmin},{vmax}:overlay_opacity=0.5: -view {view} -ras -40 -20 -20 -cc -ss "{ss_dir}{group}_{contrast}_t-map.png" -viewsize {viewsize}\n'
        os.system(cmd)
        print(cmd)

In [ ]:
# visualizations with nilearn

# define surface template
fsaverage = fetch_surf_fsaverage(mesh='fsaverage')

# loop through groups and contrasts to plot group maps
for group_indx in range(len(sub_groups)):
    group_data = all_t_maps[group_indx]
    for contrast_indx, contrast in enumerate(contrasts):
        contrast_data = group_data[contrast_indx]
        
        # create figure with multiple axes for multiple views
        fig = plt.figure(figsize=(8,6))
        ax1 = fig.add_subplot(1, 2, 1, projection='3d')
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')

        # plot the lateral view
        plotting.plot_surf_stat_map(fsaverage.infl_left, contrast_data, hemi='left',
                                    threshold = vmin, vmax = vmax,
                                    colorbar=False,
                                    bg_map=fsaverage.sulc_left,
                                    axes = ax1)
        #plot the ventral view
        plotting.plot_surf_stat_map(fsaverage.infl_left, contrast_data, hemi='left', view = 'ventral',
                                    threshold = vmin, vmax = vmax,
                                    colorbar=False,
                                    bg_map=fsaverage.sulc_left,
                                    axes = ax2)
        
        # set angles of the surfaces
        ax1.view_init(0, 180)
        ax2.view_init(270, 270)

        # add title
        fig.suptitle(f'{group_names[group_indx]} {contrast}',y = 1.0, fontsize = 'xx-large')
        
        # set dimensions of the plots
        fig.tight_layout(h_pad=-5,w_pad=0)

        # save figures
        fig.savefig(f'{fig_dir}{group_names[group_indx]}_{contrast}_{vmin}_tmaps.png')
        print('saved fig')